# <span style="color: lightblue;">Ejemplo de uso de la librería IDKRom sobre un dataset de inputs y outputs en formato csv proveniente de un modelo de IDKFem.</span>

1. Importamos los datos 

In [35]:
import sys
import os

# Agrega la carpeta raíz (un nivel arriba de 'notebooks')
sys.path.append(os.path.abspath('../..')) # para añadir idkROM al path
print(os.path.abspath('../..'))
sys.path.append(os.path.abspath('../')) # para añadir src al path
print(os.path.abspath('../'))

# Verifica la ruta actual
print("Rutas en sys.path:")
for path in sys.path:
    print(path)


d:\
d:\idkROM
Rutas en sys.path:
C:\Users\aalvarezsanz\AppData\Local\Programs\Python\Python312\python312.zip
C:\Users\aalvarezsanz\AppData\Local\Programs\Python\Python312\DLLs
C:\Users\aalvarezsanz\AppData\Local\Programs\Python\Python312\Lib
C:\Users\aalvarezsanz\AppData\Local\Programs\Python\Python312
d:\idkROM\idkROM\virtual_env

d:\idkROM\idkROM\virtual_env\Lib\site-packages
d:\idkROM\idkROM\virtual_env\Lib\site-packages\win32
d:\idkROM\idkROM\virtual_env\Lib\site-packages\win32\lib
d:\idkROM\idkROM\virtual_env\Lib\site-packages\Pythonwin
d:\idkROM
d:\idkROM\idkROM
d:\idkROM\idkROM\virtual_env\Lib\site-packages\setuptools\_vendor
d:\
d:\idkROM
d:\
d:\idkROM


In [36]:
import sys
import os
import joblib
import pandas as pd
from PyQt6.QtWidgets import QApplication
#from src.gui.main_window import ROMApp  # Asegúrate de que este archivo es el correcto
from idkROM.src.loader.import_data import DataLoader as idkROMDataLoader
from idkROM.src.pre.preprocessing import Pre as idkROMPreprocessing
from idkROM.src.models.neural_network import NeuralNetworkROM as idkROMNeuralNetwork
from idkROM.src.models.gaussian_process import GaussianProcessROM as idkROMGaussianProcess
from idkROM.src.models.rbf import RBFROM as idkROMRBF
from idkROM.src.models.polynomial_response_surface import PolynomialResponseSurface as idkROMPolynomialResponseSurface
from idkROM.src.models.svr import SVRROM as idkROMSVR
from idkROM.src.tools.search_hyperparams import search_best_hyperparameters as idkROMHyperparameterTuner
from idkROM.src.visualization.metrics import ModelReportGenerator as idkROMMetrics
from sklearn.metrics import mean_squared_error, r2_score



Cargamos el dataset como dataframe

In [37]:

file_path = "D:\\idkROM\\idkROM\\data\\main_HydrTable_DOE_Results_0601_1200 (2).csv"
data_source = "raw"
df = idkROMDataLoader().load_data(file_path, data_source)
print(f"El dataset es de dimension {df.shape}")
print(df)

El dataset es de dimension (600, 25)
        l_r_i     l_r_e  apert_pad_i   c_pad_i  apert_pad_e   c_pad_e  \
0    1.468817  0.286561    39.416713  0.033561    13.996338  0.037757   
1    0.875406  1.419291    39.837686  0.032805    17.993482  0.031606   
2    1.458757  1.451980    39.340334  0.032473    17.928236  0.027797   
3    1.480852  1.487696    31.702992  0.044865    10.476735  0.030013   
4    0.661810  0.251605    39.641655  0.032871    13.872995  0.043076   
..        ...       ...          ...       ...          ...       ...   
595  0.877197  1.467791    39.191633  0.044232    10.355015  0.025288   
596  1.074398  0.252414    36.717214  0.031243    14.387166  0.044293   
597  1.125762  0.519053    32.396154  0.031727    12.135729  0.026094   
598  0.408424  0.254989    31.965778  0.033707    17.722209  0.025003   
599  0.315469  0.860937    33.204750  0.029113    14.326586  0.043415   

        p_pieza    F_total_int   F_total_ext  Q_total_int  ...       P_sd_int  \
0    

2. Spliteamos el dataset con las proporciones deseadas

hay que tener en cuenta que nuestro dataset tiene 7 variables de entrada y 18 de salida, lo cual puede variar en función del dataset

In [38]:
last_input_var = 7
X_train, y_train, X_val, y_val, X_test, y_test = idkROMPreprocessing().split_dataset(df, last_input_var) # añadir proporciones de train, validation y test manualmente (70%, 15% y 15%)

print("Las dimensiones de cada subconjunto son:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

Las dimensiones de cada subconjunto son:
X_train: (420, 7)
y_train: (420, 18)
X_val: (90, 7)
y_val: (90, 18)
X_test: (90, 7)
y_test: (90, 18)


3. Para acabar el preprocesamiento, filtramos los outliers y aplicamos un scaler a los inputs y otro a los outputs.

In [39]:

# para decirle al script que busque en la carpeta d:\idkROM\idkROM\results, y no en la carpeta d:\idkROM\idkROM\notebooks\results (necesario para encontrar el scaler en filter_and_scale)
original_cwd = os.getcwd()

os.chdir(os.path.abspath('../'))

(X_train_normalized, y_train_normalized,
        X_val_normalized, y_val_normalized,
        X_test_normalized, y_test_normalized) = idkROMPreprocessing().filter_and_scale(X_train, y_train, X_val, y_val,
                                                                             X_test, y_test, scaler_type='minmax')

print(f"X_train_normalized: {X_train_normalized.shape}")



Original minimums of the outputs: [ 8.62532765e+04  3.35811190e+05 -6.16181137e-08  3.04891152e-05
  1.43860812e+05  1.42444489e+05  3.15157452e+05  2.66705241e+05
  1.48387697e+05  1.27600643e+05  8.14512448e-06  1.00000000e-06
  1.19024249e-05  2.84012201e-05  1.65790696e-05  3.26754884e-05
  1.64703568e-07  2.77592536e-07]
Original maximums of the outputs: [6.30155753e+05 1.61010055e+06 2.28776982e-04 5.77530561e-04
 9.81931637e+05 7.51638249e+05 3.52748811e+06 2.42088211e+07
 1.22648041e+06 1.91670601e+06 9.54379552e-05 9.86070528e-05
 9.66085958e-05 1.01925486e-04 9.76537755e-05 1.19740633e-04
 6.92199520e-06 2.57761682e-05]
Preprocessed data saved.
X_train_normalized: (419, 7)


4. Selección de hiperparametros

De momento utilizamos una funcion search_best_hyperparameters. Proporcionamos un set de rangos de hiperparametros para cada modelo, y en funcion del modelo que elijamos, la funcion llevará a cabo una randomized_search para encontrar el conjunto de hiperparametros que de el error más bajo.

4.1 Neural network

4.1.1 NN Hyperparams

In [40]:
# Definimos el espacio de hiperparametros

# Esto se hará mediante una lectura del .yml, y no manualmente ######################################
# Definir el espacio de hiperparámetros
param_grid = {
    'hidden_layers': [1, 2, 4],
    'neurons_per_layer': [5, 10, 20],
    'learning_rate': [1e-3, 1e-4],
    'activation_function': ['ReLU', 'Tanh', 'Sigmoid'],
    'optimizer': ['Adam', 'SGD', 'RMSprop'],
    'num_epochs': [1000]
}

best_params = idkROMHyperparameterTuner('neural_network', X_train_normalized, y_train_normalized, search_type='random', n_iter=10)

Shape of X_train: (419, 7)
Shape of y_train: (419, 18)
Epoch 0/1000, Training Loss: 0.236549, Validation Loss: 0.207289
Epoch 100/1000, Training Loss: 0.025462, Validation Loss: 0.016659
Epoch 200/1000, Training Loss: 0.016776, Validation Loss: 0.011360
Epoch 300/1000, Training Loss: 0.015844, Validation Loss: 0.010621
Epoch 400/1000, Training Loss: 0.015530, Validation Loss: 0.010060
Epoch 500/1000, Training Loss: 0.014754, Validation Loss: 0.010149
Epoch 600/1000, Training Loss: 0.014641, Validation Loss: 0.009847
Epoch 700/1000, Training Loss: 0.013823, Validation Loss: 0.009872
Epoch 800/1000, Training Loss: 0.014045, Validation Loss: 0.009576
Epoch 900/1000, Training Loss: 0.014032, Validation Loss: 0.009159
Modelo guardado en: d:\idkROM\results\neural_network\neural_network_model.pth
Epoch 0/1000, Training Loss: 0.256323, Validation Loss: 0.229763
Epoch 100/1000, Training Loss: 0.020000, Validation Loss: 0.012904
Epoch 200/1000, Training Loss: 0.015564, Validation Loss: 0.010599


4.1.2 Neural network training and evaluation

In [41]:
# Usamos el set de hiperparametros encontrado por la funcion de busqueda para entrenar el modelo
model = idkROMNeuralNetwork(**best_params)
model.train(X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized)



Epoch 0/1000, Training Loss: 0.129775, Validation Loss: 0.093331
Epoch 100/1000, Training Loss: 0.009791, Validation Loss: 0.003658
Epoch 200/1000, Training Loss: 0.007454, Validation Loss: 0.003192
Epoch 300/1000, Training Loss: 0.007004, Validation Loss: 0.002732
Epoch 400/1000, Training Loss: 0.006615, Validation Loss: 0.002569
Epoch 500/1000, Training Loss: 0.006753, Validation Loss: 0.002543
Epoch 600/1000, Training Loss: 0.006355, Validation Loss: 0.002298
Epoch 700/1000, Training Loss: 0.005944, Validation Loss: 0.002127
Epoch 800/1000, Training Loss: 0.006137, Validation Loss: 0.002220
Epoch 900/1000, Training Loss: 0.005863, Validation Loss: 0.002097
Modelo guardado en: d:\idkROM\results\neural_network\neural_network_model.pth


In [42]:
# Con el modelo entrenado, podemos realizar una predicción de los resultados sobre el set de test
y_pred = model.predict(X_test_normalized)
print(f"Predicciones: {y_pred}")


Predicciones: [[0.5854428  0.34500045 0.51629937 ... 0.17165485 0.1545065  0.05631291]
 [0.69881535 0.55603814 0.23599252 ... 0.09467545 0.23572907 0.07421581]
 [0.67304975 0.4839198  0.46341532 ... 0.18035279 0.16427168 0.07185655]
 ...
 [0.09417799 0.12429795 0.2958917  ... 0.52786875 0.01359496 0.03952385]
 [0.3747157  0.2546327  0.18790378 ... 0.20250262 0.15446803 0.04783724]
 [0.5379052  0.62156034 0.12855868 ... 0.37629843 0.20506775 0.08858328]]


In [43]:
# Ahora, evaluamos el error de la predicción

# Cargamos el scaler de salida guardado en el preprocesamiento
results_path = os.path.dirname(os.path.dirname(file_path))
results_path = os.path.join(results_path, "results")
output_scaler = joblib.load(os.path.join(results_path, 'output_scaler.pkl'))

if idkROMPreprocessing().scaler_type == 'minmax':
    print("Mínimos originales de los outputs:", output_scaler.data_min_)
    print("Máximos originales de los outputs:", output_scaler.data_max_)

# Evaluar el modelo utilizando las predicciones
model.evaluate(X_test_normalized, y_test_normalized, y_pred, output_scaler=output_scaler)

    

Verificación de que y_test y y_pred tengan la misma forma:
Forma de y_test: (90, 18)
Forma de y_pred: (90, 18)
MSE en escala normalizada: 0.0026856951881200075
MSE en escala original: 6317092864.0
Predicciones guardadas en: d:\idkROM\results\neural_network\neural_network_predictions.csv
Report saved at: d:\idkROM\results\neural_network\neural_network_training_summary.html
Model and metrics saved successfully!


0

In [46]:
# la variable model es el ROM Neural Network
report_generator = idkROMMetrics(model, idkROMNeuralNetwork().train_losses, idkROMNeuralNetwork().val_losses, X_train_normalized, y_train_normalized, X_test_normalized, y_test_normalized, 'neural_network')

mse = mean_squared_error(y_test_normalized, y_pred)
r2 = r2_score(y_test_normalized, y_pred)

metrics_df = pd.DataFrame([{'MSE': mse, 'R²': r2}])

report_generator.create_convergence_graph()
report_generator.create_metrics_graph(metrics_df)

os.chdir(original_cwd)



4.2 Gaussian Process

4.2.1 Gaussian Process Hyperparams

In [47]:
# Definimos el espacio de hiperparametros

# Esto se hará mediante una lectura del .yml, y no manualmente ######################################
# Definir el espacio de hiperparámetros
param_grid = {
            'kernel': ['RBF', 'Matern'],
            'noise': [1e-3, 1e-2, 1e-1],
        }

best_params = idkROMHyperparameterTuner('gaussian_process', X_train_normalized, y_train_normalized, search_type='random', n_iter=10)

Shape of X_train: (419, 7)
Shape of y_train: (419, 18)
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idk

d:\idkROM\idkROM\virtual_env\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning:

The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.



Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl
Model 

4.2.2 Gaussian process training and evaluation

In [48]:
# Usamos el set de hiperparametros encontrado por la funcion de busqueda para entrenar el modelo
model = idkROMGaussianProcess(**best_params)
model.train(X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized)



Model saved at: d:\idkROM\idkROM\results\gaussian_process\gp_model.pkl


In [49]:
# Con el modelo entrenado, podemos realizar una predicción de los resultados sobre el set de test
y_pred = model.predict(X_test_normalized)
print(f"Predicciones: {y_pred}")


Predicciones: [[ 0.59379499  0.30266087  0.45958701 ...  0.15781325  0.15174953
   0.03453482]
 [ 0.7467815   0.54681845  0.19843033 ...  0.11555533  0.20072701
   0.13029213]
 [ 0.68514549  0.51602485  0.42281735 ...  0.26474846  0.19088709
   0.10297304]
 ...
 [ 0.06619488  0.10937262  0.25476329 ...  0.49477316 -0.01345827
   0.02442281]
 [ 0.36073756  0.23800714  0.14100647 ...  0.1890957   0.11991487
   0.02840378]
 [ 0.56197347  0.63778301  0.19897238 ...  0.37049591  0.21413023
   0.10043926]]


In [50]:
# Ahora, evaluamos el error de la predicción

# Cargamos el scaler de salida guardado en el preprocesamiento
results_path = os.path.dirname(os.path.dirname(file_path))
results_path = os.path.join(results_path, "results")
output_scaler = joblib.load(os.path.join(results_path, 'output_scaler.pkl'))

if idkROMPreprocessing().scaler_type == 'minmax':
    print("Mínimos originales de los outputs:", output_scaler.data_min_)
    print("Máximos originales de los outputs:", output_scaler.data_max_)

# Evaluar el modelo utilizando las predicciones
model.evaluate(X_test_normalized, y_test_normalized, y_pred, output_scaler=output_scaler)

    

Predictions saved at: d:\idkROM\idkROM\results\gaussian_process\gaussian_process_predictions.csv
MSE en escala normalizada: 0.0008132043030541896
MSE en escala original: 41026878691.087555
Report saved at: d:\idkROM\idkROM\results\gaussian_process\gaussian_process_training_summary.html
Model and metrics saved successfully!


np.float64(0.0008132043030541896)

In [53]:
# la variable model es el ROM Neural Network
report_generator = idkROMMetrics(model, None, None, X_train_normalized, y_train_normalized, X_test_normalized, y_test_normalized, 'gaussian_process')

mse = mean_squared_error(y_test_normalized, y_pred)
r2 = r2_score(y_test_normalized, y_pred)

metrics_df = pd.DataFrame([{'MSE': mse, 'R²': r2}])

report_generator.create_convergence_graph()
report_generator.create_metrics_graph(metrics_df)

os.chdir(original_cwd)



4.3 Polynomial Response Surface

4.3.1 PRS Hyperparams

In [54]:
# Definimos el espacio de hiperparametros

# Esto se hará mediante una lectura del .yml, y no manualmente ######################################
# Definir el espacio de hiperparámetros
param_grid = {
            'degree': [2, 3, 4, 5, 6, 7]
        }

best_params = idkROMHyperparameterTuner('response_surface', X_train_normalized, y_train_normalized, search_type='random', n_iter=10)

Shape of X_train: (419, 7)
Shape of y_train: (419, 18)
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl


d:\idkROM\idkROM\virtual_env\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning:

The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.



Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl
Model saved at: d:\idkROM\idkROM\results\response_su

4.1.2 Response surface training and evaluation

In [55]:
# Usamos el set de hiperparametros encontrado por la funcion de busqueda para entrenar el modelo
model = idkROMPolynomialResponseSurface(**best_params)
model.train(X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized)



Model saved at: d:\idkROM\idkROM\results\response_surface\polynomial_model.pkl


In [56]:
# Con el modelo entrenado, podemos realizar una predicción de los resultados sobre el set de test
y_pred = model.predict(X_test_normalized)
print(f"Predicciones: {y_pred}")


Predicciones: [[ 0.57397922  0.31996446  0.43897663 ...  0.1858047   0.16481068
   0.05533656]
 [ 0.80534645  0.58635671  0.20999492 ...  0.14250137  0.27766246
   0.17660745]
 [ 0.67565455  0.51490002  0.42244885 ...  0.2619101   0.18987413
   0.10996449]
 ...
 [ 0.03853296  0.12620571  0.29421301 ...  0.51336854 -0.01329701
   0.03065587]
 [ 0.3685445   0.25546459  0.14432923 ...  0.22435942  0.16233843
   0.0717781 ]
 [ 0.56119599  0.65956136  0.19252998 ...  0.39281345  0.20902004
   0.10350895]]


In [57]:
# Ahora, evaluamos el error de la predicción

# Cargamos el scaler de salida guardado en el preprocesamiento
results_path = os.path.dirname(os.path.dirname(file_path))
results_path = os.path.join(results_path, "results")
output_scaler = joblib.load(os.path.join(results_path, 'output_scaler.pkl'))

if idkROMPreprocessing().scaler_type == 'minmax':
    print("Mínimos originales de los outputs:", output_scaler.data_min_)
    print("Máximos originales de los outputs:", output_scaler.data_max_)

# Evaluar el modelo utilizando las predicciones
model.evaluate(X_test_normalized, y_test_normalized, y_pred, output_scaler=output_scaler)

    

Predictions saved at: d:\idkROM\idkROM\results\response_surface\response_surface_predictions.csv
MSE in normalized scale: 0.0006133694473505606
MSE in original scale: 14193917124.676876
Report saved at: d:\idkROM\idkROM\results\response_surface\response_surface_training_summary.html
Model and metrics saved successfully!


0.0006133694473505606

In [58]:
# la variable model es el ROM Neural Network
report_generator = idkROMMetrics(model, None, None, X_train_normalized, y_train_normalized, X_test_normalized, y_test_normalized, 'response_surface')

mse = mean_squared_error(y_test_normalized, y_pred)
r2 = r2_score(y_test_normalized, y_pred)

metrics_df = pd.DataFrame([{'MSE': mse, 'R²': r2}])

report_generator.create_convergence_graph()
report_generator.create_metrics_graph(metrics_df)

os.chdir(original_cwd)



4.4 Radial Basis Function

4.4.1 RBF Hyperparams

In [59]:
# Definimos el espacio de hiperparametros

# Esto se hará mediante una lectura del .yml, y no manualmente ######################################
# Definir el espacio de hiperparámetros
param_grid = {
            'gamma': [0.1, 1.0, 10.0]
        }

best_params = idkROMHyperparameterTuner('rbf', X_train_normalized, y_train_normalized, search_type='random', n_iter=10)

Shape of X_train: (419, 7)
Shape of y_train: (419, 18)
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl


d:\idkROM\idkROM\virtual_env\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning:

The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.



Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl
Mejores hiperparámetros encontrados:  {'gamma': 1.0}
Mejor score encontrado:  -0.002247244455683613


4.4.2 RBF training and evaluation

In [60]:
# Usamos el set de hiperparametros encontrado por la funcion de busqueda para entrenar el modelo
model = idkROMRBF(**best_params)
model.train(X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized)



Model saved at: d:\idkROM\idkROM\results\rbf\rbf_model.pkl


In [61]:
# Con el modelo entrenado, podemos realizar una predicción de los resultados sobre el set de test
y_pred = model.predict(X_test_normalized)
print(f"Predicciones: {y_pred}")


Predicciones:     F_total_int  F_total_ext  Q_total_int  Q_total_ext  P_mean_int  \
0      0.574548     0.294496     0.445211     0.086681    0.646906   
1      0.736252     0.543547     0.190913    -0.024752    0.846969   
2      0.676841     0.510532     0.426832     0.135991    0.817653   
3      0.351524     0.272137     0.559048     0.674740    0.431883   
4      0.174609     0.098046     0.127080     0.141426    0.195130   
..          ...          ...          ...          ...         ...   
85     0.234432     0.131253     0.154694     0.172408    0.299481   
86     0.684870     0.610473     0.113445     0.056864    0.698522   
87     0.071828     0.114082     0.271693     0.414529    0.056712   
88     0.365924     0.248211     0.129160     0.081059    0.394293   
89     0.560349     0.637317     0.195054     0.382653    0.708365   

    P_mean_ext  P_max_int  P_max_ext  P_sd_int  P_sd_ext  h_min_int  \
0     0.295544   0.442121  -0.000991  0.564782  0.111982   0.559002   
1  

In [62]:
# Ahora, evaluamos el error de la predicción

# Cargamos el scaler de salida guardado en el preprocesamiento
results_path = os.path.dirname(os.path.dirname(file_path))
results_path = os.path.join(results_path, "results")
output_scaler = joblib.load(os.path.join(results_path, 'output_scaler.pkl'))

if idkROMPreprocessing().scaler_type == 'minmax':
    print("Mínimos originales de los outputs:", output_scaler.data_min_)
    print("Máximos originales de los outputs:", output_scaler.data_max_)

# Evaluar el modelo utilizando las predicciones
model.evaluate(X_test_normalized, y_test_normalized, y_pred, output_scaler=output_scaler)

    

Predictions saved at: d:\idkROM\idkROM\results\rbf\rbf_predictions.csv
MSE in normalized scale: 0.0015787506811099851
MSE in original scale: 82229559784.62984
Report saved at: d:\idkROM\idkROM\results\rbf\rbf_training_summary.html
Model and metrics saved successfully!


0.0015787506811099851

In [63]:
# la variable model es el ROM Neural Network
report_generator = idkROMMetrics(model, None, None, X_train_normalized, y_train_normalized, X_test_normalized, y_test_normalized, 'rbf')

mse = mean_squared_error(y_test_normalized, y_pred)
r2 = r2_score(y_test_normalized, y_pred)

metrics_df = pd.DataFrame([{'MSE': mse, 'R²': r2}])

report_generator.create_convergence_graph()
report_generator.create_metrics_graph(metrics_df)

os.chdir(original_cwd)



4.5 SVR

4.5.1 SVR Hyperparams

In [67]:
# Definimos el espacio de hiperparametros

# Esto se hará mediante una lectura del .yml, y no manualmente ######################################
# Definir el espacio de hiperparámetros
param_grid = {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [0.1, 5, 50],
            'epsilon': [0.1, 0.2, 0]
        }

# no funciona para SVR
# best_params = idkROMHyperparameterTuner('svr', X_train_normalized, y_train_normalized, search_type='random', n_iter=10)


4.5.2 SVR training and evaluation

/